In [1]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse
from langchain import LLMChain
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline

/home/ubuntu/anaconda3/envs/llm/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
!pip3 install auto-gptq==0.2.2

  Using cached auto_gptq-0.2.2.tar.gz (52 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [105 lines of output]
      appending conda cuda include dir /home/ubuntu/anaconda3/envs/llm/lib/python3.10/site-packages/nvidia/cuda_runtime/include
      include dirs are: ['autogptq_cuda', '/home/ubuntu/anaconda3/envs/llm/lib/python3.10/site-packages/nvidia/cuda_runtime/include']
      running bdist_wheel
      /home/ubuntu/anaconda3/envs/llm/lib/python3.10/site-packages/torch/utils/cpp_extension.py:500: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
        warnings.warn(msg.format('we could not find ninja.'))
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/auto_gpt

In [3]:
model = "meta-llama/Llama-2-7b-chat-hf" # meta-llama/Llama-2-7b-chat-hf

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/home/ubuntu/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:732: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [4]:
from transformers import pipeline
import torch 

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    temperature=0.1,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
template = """
[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
here is the chat history
{chat_history}

{prompt} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "prompt"],
    template=template
)

In [6]:
llm=HuggingFacePipeline(pipeline=llama_pipeline)
memory = ConversationBufferMemory(memory_key="chat_history",    k=3,
    return_messages=True)

In [7]:
llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=False,  memory=memory)


In [8]:
# while 1:
#   text=input("You: ")
#   if text=='end':
#     break
#   output=llm_chain.predict(prompt=text)
#   print("Chatbot: ",output)

In [9]:
output=llm_chain.predict(prompt="Hello")
print(output.split("[/INST]")[-1])


Hello! I'm here to help you with any questions or concerns you may have. Please feel free to ask me anything, and I will do my best to provide helpful and respectful responses. I'm programmed to be socially unbiased and positive in nature, and I will always strive to ensure that my answers are safe and free from harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. If a question does not make sense or is not factually coherent, I will explain why instead of answering something not correct. If I don't know the answer to a question, I will not share false information. Is there anything specific you would like to know or discuss?


In [11]:
def predict(inputs, history):

  response = llm_chain.predict(prompt=inputs)
  print(response.split("[/INST]")[-1])
  return response.split("[/INST]")[-1]

In [12]:
import gradio as gr

demo = gr.ChatInterface(fn=predict, title="Echo Bot")
demo.launch(share=True,debug = True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://310daa06be60c819fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)



Hello! It's nice to chat with you. Is there something specific you would like to talk about or ask? I'm here to help with any questions or concerns you may have. Please feel free to ask me anything, and I will do my best to provide helpful and respectful responses.

Hello! I'm here to help you with any questions or concerns you may have. Is there something specific you need help with? Please feel free to ask me anything, and I will do my best to provide helpful and respectful responses. Is there something specific you would like to know or discuss?


Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/llm/lib/python3.10/site-packages/gradio/queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
  File "/home/ubuntu/anaconda3/envs/llm/lib/python3.10/site-packages/gradio/route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/ubuntu/anaconda3/envs/llm/lib/python3.10/site-packages/gradio/blocks.py", line 1741, in process_api
    result = await self.call_function(
  File "/home/ubuntu/anaconda3/envs/llm/lib/python3.10/site-packages/gradio/blocks.py", line 1294, in call_function
    prediction = await fn(*processed_input)
  File "/home/ubuntu/anaconda3/envs/llm/lib/python3.10/site-packages/gradio/utils.py", line 718, in async_wrapper
    response = await f(*args, **kwargs)
  File "/home/ubuntu/anaconda3/envs/llm/lib/python3.10/site-packages/gradio/chat_interface.py", line 507, in _submit_fn
    response = await anyio.to_thread.

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://310daa06be60c819fa.gradio.live
